In [1]:
from transformers import MBartForConditionalGeneration

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from indobenchmark import IndoNLGTokenizer

tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")

In [9]:
model = MBartForConditionalGeneration.from_pretrained("haryoaw/lexicol-indobart-v2")

In [ ]:
summary_ids = model.generate(inputs['input_ids'], decoder_input_ids = decoder_inputs['inputs_ids']  num_beams=4, max_length=5, early_stopping=True)

In [5]:
inp = 'min biaya gw dong, thx'

inp_to_model = tokenizer(inp, return_tensors='pt')['input_ids']
result = model.generate(
    inp_to_model, num_beams=8, max_length=100
)

In [8]:
tokenizer.decode(result[0])

'</s> admin , biaya saya , terima kasih . terima kasih biaya seperti admin . admin , admin . min . admin . saya biaya saya biaya biaya biaya admin . . . hahaha . . tolong biaya saya . . admin ? terima kasih , biaya seperti ini saya biaya min . min , biayax . . ya min . biaya admin , , biaya terima kasih min . . min min . ya admin . telepon admin , menggunakan menggunakan saya biaya nanti , biaya datang , admin , bukan bukan ? admin . kasih .</s>'

In [5]:
import pandas as pd

In [10]:
df = pd.read_csv("../data/scolid/test_with_pointing.csv")

In [11]:
from tqdm import tqdm

In [12]:
inf_result = []
for inf in tqdm(df.informal):
    inp_to_model = tokenizer(inf, return_tensors='pt')['input_ids']
    result = model.generate(
        inp_to_model, num_beams=10, max_length=100
    )
    inf_result.append(tokenizer.decode(result[0], skip_special_tokens=True))

100%|██████████| 245/245 [10:57<00:00,  2.68s/it]


In [13]:
inf_result

[' jd?! apa maksud anda dengan semua komentar-komentar yang ada ini??????',
 ' berasa silauuuuuuuuu',
 ' baru aku sadar dia cantik beudd @dianputri929 diantak termiribkankankan tubuh @duddewi929',
 ' happy birthday kak, semoga bahagia selalu dan segera married. tuhan memberkati semua aspek dalam kehidupanmu. ?? ?? ??',
 ' uda menonton kak disurabaya disurabaya surabaya disurabaya.',
 ' god is love ? itu dkuburnya daerah mana iya.. iya..',
 ' amiin ya allah allah allah amin ya allah amin amin yaallah ya allah aamiin ya allah ya allah amiin amiin allah amin amiin yaallah amin amin allah amiin ya amin amin amin amiin amin ya amin allah amin allah allah amiin amin amin aamiin yaallah amiin ya ya allahamin ya allah alah amiin amiin amin amiin amiin ya alah amin amiin allah amiin allah allah swt ya allah doanya ya allah semoga ya allah terkabul ya allahallah amin ya amiin ya rabb ya allah ya allah kabul amin ya alah amiin ya',
 ' @miaznr awas lo enggak konsisten',
 ' gak menor n norak!! engg

In [16]:
# output it to csv
df['formal_predict'] = inf_result
df.to_csv("../data/pred/lexicol_indobart.csv", index=False)

In [15]:
# use sacrebleu
from sacrebleu import corpus_bleu

## detokenize

corpus_bleu(inf_result, [df.formal.to_list()], lowercase=True)

BLEU = 48.88 58.3/51.8/46.0/41.1 (BP = 1.000 ratio = 1.427 hyp_len = 5939 ref_len = 4161)

In [17]:
from evaluate import load
bertscore = load("bertscore")
predictions = inf_result
references = df.formal.to_list()
results = bertscore.compute(predictions=predictions, references=references, lang="id")

In [18]:
predictions = inf_result
references = df.formal.to_list()
results = bertscore.compute(predictions=predictions, references=references, lang="id")

In [19]:
import numpy as np

np.mean(results['f1'])

0.6861076844296836